# Fire Detection Using Transfer Learning

## Importing Libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from datetime import datetime

## Loading Dataset

In [ ]:
BASE_DIR = "fire_dataset/"


FIRE_DIR = os.path.join(BASE_DIR, "fire_images")
NONFIRE_DIR = os.path.join(BASE_DIR, "non_fire_images")

print("🔥 Fire images:", len(os.listdir(FIRE_DIR)))
print("🌿 Non-fire images:", len(os.listdir(NONFIRE_DIR)))


## Data Preprocessing and Image Data Generator

In [ ]:
IMG_SIZE = (224, 224)     # Required for EfficientNet/MobileNet
BATCH_SIZE = 32

In [ ]:
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.20
)

train_gen = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

In [ ]:
idx_to_label = {v: k for k, v in train_gen.class_indices.items()}
idx_to_label

## Model Compilation and Class Weight Balancing

In [ ]:
y_classes = train_gen.classes

weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_classes),
    y=y_classes
)

class_weights = {i: w for i, w in enumerate(weights)}
class_weights

## Transfer Learning with EfficientNetB0

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    input_shape=(224,224,3),
    weights='imagenet'
)
base_model.trainable = False   # first stage

## Model Architecture: Transfer Learning

In [ ]:
inputs = layers.Input(shape=(*IMG_SIZE, 3))

x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


## Tensorboard Integration

In [ ]:
log_dir = "logs/effnet_fire_" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='batch',       # 🔥 more frequent logging
    profile_batch=5            # 🔥 profile performance on batch 5
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [ ]:
log_dir

## Model Training

In [ ]:
history = model.fit(
    train_gen,
    epochs=15,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=[tensorboard_callback, early_stop]
)

## Visualizations

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.legend(["Train", "Validation"])

plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.legend(["Train", "Validation"])

plt.show()


## Fine Tuning the Model

In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history_finetune = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=[tensorboard_callback]
)


## Visualizations after Fine Tuning

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history_finetune.history['accuracy'])
plt.plot(history_finetune.history['val_accuracy'])
plt.title("Model Accuracy")
plt.legend(["Train", "Validation"])

plt.subplot(1,2,2)
plt.plot(history_finetune.history['loss'])
plt.plot(history_finetune.history['val_loss'])
plt.title("Model Loss")
plt.legend(["Train", "Validation"])

plt.show()


In [ ]:
model

In [ ]:
val_gen_cm = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

y_true = val_gen_cm.classes
y_pred_prob = model.predict(val_gen_cm)
y_pred = (y_pred_prob > 0.5).astype("int32")

cm = confusion_matrix(y_true, y_pred)

import seaborn as sns
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=idx_to_label.values(),
            yticklabels=idx_to_label.values())
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

print(classification_report(y_true, y_pred,
      target_names=list(idx_to_label.values())))


## Performance Improvement Using another Pre-Trained model

## Transfer Learning with VGG16

In [ ]:
import tensorflow as tf
from tensorflow import keras
keras.applications.VGG16(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
    name="vgg16",
)

In [ ]:
inputs = layers.Input(shape=(*IMG_SIZE, 3))

x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)

model = models.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
log_dir = "logs/VGG16_fire_" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='batch',       # 🔥 more frequent logging
    profile_batch=5            # 🔥 profile performance on batch 5
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [ ]:
log_dir

In [ ]:
history = model.fit(
    train_gen,
    epochs=15,
    validation_data=val_gen,
    class_weight=class_weights,
    callbacks=[tensorboard_callback, early_stop]
)

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.legend(["Train", "Validation"])

plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.legend(["Train", "Validation"])

plt.show()

In [ ]:
val_gen_cm = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

y_true = val_gen_cm.classes
y_pred_prob = model.predict(val_gen_cm)
y_pred = (y_pred_prob > 0.5).astype("int32")

cm = confusion_matrix(y_true, y_pred)

import seaborn as sns
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=idx_to_label.values(),
            yticklabels=idx_to_label.values())
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

print(classification_report(y_true, y_pred,
      target_names=list(idx_to_label.values())))

## Testing the Model

In [ ]:
from tensorflow.keras.preprocessing import image

def predict_image(path):
    img = image.load_img(path, target_size=IMG_SIZE)
    img_arr = image.img_to_array(img) / 255.0
    img_arr = np.expand_dims(img_arr, axis=0)

    prob = model.predict(img_arr)[0][0]
    pred_class = int(prob > 0.5)

    label = idx_to_label[pred_class]

    plt.imshow(img)
    plt.axis("off")

    if "fire" in label:
        plt.title(f"🔥 FIRE DETECTED (prob={prob:.2f})")
    else:
        plt.title(f"🌿 NON-FIRE (prob={prob:.2f})")

    plt.show()


## Saving Model

In [ ]:
model.save("fire_detection_cnn")